In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format('json')\
        .option('inferSchema', 'true')\
        .option('multiline', 'true')\
        .load('/Volumes/parent_cata/streaming/stream/datasource/')

display(df)

In [0]:
df_flat = df.select(
    'order_id',
    'timestamp',
    'customer.customer_id',
    'customer.name',
    'customer.email',
    'customer.address.city',
    'customer.address.country',
    'customer.address.postal_code',
    'payment.method',
    'payment.transaction_id',
    'items',
    'metadata'
)


df_flat.display()

# **items explode**

In [0]:
df_items = df_flat.withColumn('items',explode('items'))

df_items = df_items.select(
    'order_id',
    'timestamp',
    'customer_id',
    'name',
    'email',
    'city',
    'country',
    'postal_code',
    'method',
    'transaction_id',
    'items.item_id',
    'items.price',
    'items.product_name',
    'items.quantity',
    'metadata'
)

display(df_items)

In [0]:
df_final = df_items.withColumn("meta", explode(col("metadata"))).select(
    "*",
    col("meta.key").alias("meta_key"),
    col("meta.value").alias("meta_value")
).drop("meta", "metadata")

display(df_final)

In [0]:
df_final.write.format('delta')\
    .mode('append')\
    .save('/Volumes/parent_cata/streaming/stream/datasink/')

In [0]:
%sql
select * from delta.`/Volumes/parent_cata/streaming/stream/datasink/`